In [10]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
import boto3
import pandas as pd
!pip install s3fs

    100% |████████████████████████████████| 71kB 4.3MB/s ta 0:00:011
    100% |████████████████████████████████| 133kB 9.7MB/s eta 0:00:01
    100% |████████████████████████████████| 5.9MB 4.3MB/s eta 0:00:01    27% |████████▊                       | 1.6MB 25.5MB/s eta 0:00:01    71% |██████████████████████▉         | 4.2MB 29.3MB/s eta 0:00:01    96% |███████████████████████████████ | 5.7MB 33.2MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 17.0MB/s ta 0:00:01
awscli 1.16.17 has requirement botocore==1.12.7, but you'll have botocore 1.14.8 which is incompatible.
awscli 1.16.17 has requirement s3transfer<0.2.0,>=0.1.12, but you'll have s3transfer 0.3.2 which is incompatible.
  Found existing installation: botocore 1.12.7
    Uninstalling botocore-1.12.7:
      Successfully uninstalled botocore-1.12.7
  Found existing installation: s3transfer 0.1.13
    Uninstalling s3transfer-0.1.13:
      Successfully uninstalled s3transfer-0.1.13
  Found existing installation: boto

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

In [33]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [34]:
spark = create_spark_session()

In [35]:
s3 = boto3.client("s3", region_name="us-west-2", aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
                    aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'])

In [36]:
searchobj = "log_data"
lcobj = list(s3.list_objects_v2(Bucket="udacity-dend", 
                                Prefix=searchobj + "/").values())

In [37]:
sdlist = list()
prefix = 's3a://udacity-dend/'
for o in lcobj[2]:
    sdlist.append(prefix+o["Key"])

In [38]:
sdlist[1]

's3a://udacity-dend/log_data/2018/11/2018-11-01-events.json'

In [44]:
df = pd.read_json(sdlist[1], lines=True)

In [45]:
df.shape

(15, 18)

In [49]:
sdf = spark.read.json(sdlist[1])

In [51]:
sdf.show(7)

+--------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|        artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|          null|Logged In|   Walter|     M|            0|    Frye|     null| free|San Francisco-Oak...|   GET|    Home|1.540919166796E12|       38|                null|   200|1541105830796|"Mozilla/5.0 (Mac...|    39|
|          null|Logged In|   Kaylee|     F|            0| Summers|     null| free|Phoenix-Mesa-Scot...|   GET|    Home|1.5403447

In [40]:
?pd.read_json

Signature: pd.read_json(path_or_buf=None, orient=None, typ='frame', dtype=True, convert_axes=True, convert_dates=True, keep_default_dates=True, numpy=False, precise_float=False, date_unit=None, encoding=None, lines=False, chunksize=None, compression='infer')
Docstring:
Convert a JSON string to pandas object

Parameters
----------
path_or_buf : a valid JSON string or file-like, default: None
    The string could be a URL. Valid URL schemes include http, ftp, s3, and
    file. For file URLs, a host is expected. For instance, a local file
    could be ``file://localhost/path/to/table.json``

orient : string,
    Indication of expected JSON string format.
    Compatible JSON strings can be produced by ``to_json()`` with a
    corresponding orient value.
    The set of possible orients is:

    - ``'split'`` : dict like
      ``{index -> [index], columns -> [columns], data -> [values]}``
    - ``'records'`` : list like
      ``[{column -> value}, ... , {column -> value}]``
    - ``'index'``

In [ ]:
def process_song_data(spark, input_data, output_data):
    # get filepath to song data file
    song_data = 
    
    # read song data file
    df = 

    # extract columns to create songs table
    songs_table = 
    
    # write songs table to parquet files partitioned by year and artist
    songs_table

    # extract columns to create artists table
    artists_table = 
    
    # write artists table to parquet files
    artists_table